#Setup

###Imports

In [ ]:
import time as timing
import PIL
from PIL import Image
from google.colab import drive
import pandas as pd
import scipy
import os
import cv2
import pickle 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import math
from pathlib import Path
import shutil
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.models as tv_models
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
from torch.cuda import amp
import torch.nn.functional as F
from torchsummary import summary

!pip install tqdm
from tqdm import tqdm as load_bar

!pip install codecarbon
import codecarbon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 5.6 MB/s 
     |████████████████████████████████| 66 kB 5.5 MB/s 
     |████████████████████████████████| 46 kB 2.7 MB/s 


In [ ]:
if torch.cuda.is_available():
  import cupy
  device = torch.device("cuda")
  name = torch.cuda.get_device_name(0)
  device = 'cuda:0'
  print(device, name)
else:
  device = "cpu"
  print(device)

cuda:0 Tesla T4


In [ ]:
########################### LATEST VERSION INSTALL ###########################

#Currently working on the latest release - Not yet on PyPl, must clone the repo
!git clone https://github.com/fangwei123456/spikingjelly.git

#Add to the PATH the new directory
import sys
sys.path.append('/content/spikingjelly/')

#Install requirements for spikingjelly
%cd /content/spikingjelly/
!python setup.py install

#Now we can import these modules
import spikingjelly
from spikingjelly.activation_based import neuron, encoding, functional, surrogate, layer, learning
from spikingjelly import visualizing
from spikingjelly.activation_based import ann2snn
from spikingjelly.activation_based.model import spiking_vgg, spiking_resnet


Cloning into 'spikingjelly'...
remote: Enumerating objects: 12830, done.
remote: Counting objects: 100% (2649/2649), done.
remote: Compressing objects: 100% (811/811), done.
remote: Total 12830 (delta 1924), reused 2501 (delta 1826), pack-reused 10181
Receiving objects: 100% (12830/12830), 40.86 MiB | 30.25 MiB/s, done.
Resolving deltas: 100% (8526/8526), done.
/content/spikingjelly
running install
running bdist_egg
running egg_info
creating spikingjelly.egg-info
writing spikingjelly.egg-info/PKG-INFO
writing dependency_links to spikingjelly.egg-info/dependency_links.txt
writing requirements to spikingjelly.egg-info/requires.txt
writing top-level names to spikingjelly.egg-info/top_level.txt
writing manifest file 'spikingjelly.egg-info/SOURCES.txt'
adding license file 'LICENSE'
adding license file 'LICENSE-CN'
writing manifest file 'spikingjelly.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating buil

###Dirs

In [ ]:
!mkdir "/content/runs"
!mkdir "/content/emissions"
!mkdir "/content/saved_models"
!mkdir "/content/datasets"

RUNS_PATH = "/content/runs"
EMISSION_PATH = "/content/emissions"
MODEL_PATH = "/content/saved_models"
DATASET_PATH = "/content/datasets"

tracker = codecarbon.EmissionsTracker(output_dir = EMISSION_PATH, save_to_file=True)
tracker_inf = codecarbon.EmissionsTracker(output_dir = EMISSION_PATH, save_to_file=True, output_file='infe_emissions.csv')

[codecarbon INFO @ 18:05:44] [setup] RAM Tracking...
[codecarbon INFO @ 18:05:44] [setup] GPU Tracking...
[codecarbon INFO @ 18:05:44] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:05:44] [setup] CPU Tracking...
[codecarbon WARNING @ 18:05:44] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:05:47] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:05:47] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 18:05:47] >>> Tracker's metadata:
[codecarbon INFO @ 18:05:47]   Platform system: Linux-5.10.133+-x86_64-with-glibc2.27
[codecarbon INFO @ 18:05:47]   Python version: 3.8.16
[codecarbon INFO @ 18:05:47]   Available RAM : 12.681 GB
[codecarbon INFO @ 18:05:47]   CPU count: 2
[codecarbon INFO @ 18:05:47]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 18:05:47]   GPU count: 1
[codecarbon INFO @ 18:05:47]   GPU model: 1 x Te

#Datasets

In [ ]:
def init_data_loader(dataset, batch, shuffle:bool=True): 

  data_loader = data.DataLoader(
            dataset = dataset,
            batch_size = batch,
            shuffle = shuffle,
            drop_last = True)  
  
  return data_loader



def transformations (steps):

  crop = steps.crop_size[0]
  resize_s = steps.resize_size[0]
  interpolation = steps.interpolation
  
  compose = torchvision.transforms.Compose([
      torchvision.transforms.CenterCrop(crop),
      torchvision.transforms.Resize(resize_s, interpolation = interpolation),
      torchvision.transforms.ToTensor()
  ])

  return compose

###Mnist


In [ ]:
train_dataset = torchvision.datasets.MNIST(
        root = DATASET_PATH + "/" + "MNIST/train",
        train = True,
        transform = torchvision.transforms.ToTensor(),
        download = True)

test_dataset = torchvision.datasets.MNIST(
        root = DATASET_PATH + "/" + "MNIST/test",
        train = False,
        transform = torchvision.transforms.ToTensor(),
        download = True)

#Split train -> train and validation
train_dataset, validation_dataset = torch.utils.data.random_split(train_dataset, [50000, 10000])

###ImgNet

In [ ]:
drive.mount('/content/drive', force_remount=True)
MAIN_PATH = '/content/drive/MyDrive/UNIPD_Projects/Decepticon'

IMGNET_PATH = '/content/datasets/ImgNet'
!mkdir '/content/datasets/ImgNet'

shutil.copyfile(MAIN_PATH + '/test_imgs.zip', IMGNET_PATH + '/test_imgs.zip')

!mkdir '/content/datasets/ImgNet/imgs'
!unzip '/content/datasets/ImgNet/test_imgs.zip' -d  '/content/datasets/ImgNet/imgs'

IMGNET_PATH = '/content/datasets/ImgNet/imgs/test'

lis = pd.read_pickle(r'/content/drive/MyDrive/UNIPD_Projects/Decepticon/lab_dict')

with open(os.path.join(MAIN_PATH, 'label_dict.pkl'), 'rb') as f:
    lab_name_n = pickle.load(f)

In [ ]:
def get_std_pretrained(model_name):

  model = tv_models.get_model(model_name, weights="DEFAULT")
  model_weights = tv_models.get_model_weights(model_name)
  preprocess_steps = model_weights.DEFAULT.transforms()

  model_compose = transformations(preprocess_steps)

  return model, model_compose

In [ ]:
_, compose_vgg19 = get_std_pretrained('vgg19_bn')
_, compose_resnet50 = get_std_pretrained('resnet50')

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    imgnet_categories = [s.strip() for s in f.readlines()]

--2022-12-28 19:48:18--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2022-12-28 19:48:18 (84.6 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
imgnet_test = []
imgnet_test_names = []

for root, __, files in os.walk(IMGNET_PATH):
  for f in files:
    imgnet_test.append(Image.open(os.path.join(root, f)))
    imgnet_test_names.append(int(f.replace('.png', '')))

imgnet_test_names_sorted, imgnet_test_sorted = zip(*sorted(zip(imgnet_test_names, imgnet_test)))

In [ ]:
ls_vgg19 = []
ls_resnet50 = []

for a, b in zip(imgnet_test_sorted, imgnet_test_names_sorted):
  a_vgg19 = compose_vgg19(a)
  a_resnet50 = compose_resnet50(a)

  ls_vgg19.append(tuple((a_vgg19,b)))
  ls_resnet50.append(tuple((a_resnet50,b)))


In [ ]:
batch_size = 32

imgnet_vgg19_loader = init_data_loader(ls_vgg19, batch_size)
imgnet_resnet50_loader = init_data_loader(ls_resnet50, batch_size)

###Cifar10

In [ ]:
#torch.Size([3, 32, 32])
cifar_compose = torchvision.transforms.Compose([
                    torchvision.transforms.Resize((32,32)),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_cifar = torchvision.datasets.CIFAR10(
                  root = DATASET_PATH + "/" + "Cifar10/train",
                  train = True,
                  download = True, 
                  transform = cifar_compose)

test_cifar = torchvision.datasets.CIFAR10(
                  root = DATASET_PATH + "/" + "Cifar10/test",
                  train = False,
                  download = True, 
                  transform = cifar_compose)

#Split train -> train and validation
train_cifar, val_cifar = torch.utils.data.random_split(train_cifar, [.9, .1])

cifar_classes = ('plane', 'car', 'bird', 'cat',
                  'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

cifar_nclasses = len(cifar_classes)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/datasets/Cifar10/train/cifar-10-python.tar.gz to /content/datasets/Cifar10/train


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/datasets/Cifar10/test/cifar-10-python.tar.gz to /content/datasets/Cifar10/test


#Functions

###Utils functions

In [ ]:
def tensor_to_PIL (tens):
  return torchvision.transforms.ToPILImage()(tens)

In [ ]:
def plot_performance_dict(hist):

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8,4))
    axes[0].plot(hist['Train_acc'], label='train acc')
    axes[0].plot(hist['Valid_acc'], label='valid acc')
    axes[0].title.set_text('Accuracy train and val')
    axes[0].legend()

    axes[1].plot(hist['Train_loss'], label='train loss')
    axes[1].plot(hist['Valid_loss'], label='valid loss')
    axes[1].title.set_text('Loss train and val')
    axes[1].legend()

    plt.show()


def plot_performance_list(acc, loss):

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8,4))
    axes[0].plot(acc, label='train acc')
    axes[0].title.set_text('Accuracy train')

    axes[1].plot(loss, label='train loss')
    axes[1].title.set_text('Loss train')

    plt.show()

In [ ]:
def single_infer_topk(model, img, compose, idx_label, k=5, plot_img = False):

  model.to(device)
  model.eval()

  with torch.no_grad():
    tens_img = compose(img).to(device)
    out_fr = model(tens_img.unsqueeze(0))

  probabilities = torch.nn.functional.softmax(out_fr[0], dim=0)
  top5_prob, top5_catid = torch.topk(probabilities, k)

  print("GROUND TRUTH: ", lab_name_n(idx_label), idx_label)
  
  for i in range(top5_prob.size(0)):
      print(imgnet_categories[top5_catid[i]], np.round(top5_prob[i].item(),3))
  
  if plot_img == True:
    plt.imshow(img)

  if("spiking" in str(type(model)).lower()):
    functional.reset_net(model)
    print("\n -- resetted net --")

###Spiking functions

In [ ]:
def fit_and_eval(net, epochs, optimizer, criterion, num_classes, 
                 train_data_loader, validation_data_loader, device,
                 T, validation_step = False, convolution = True, onehot = False):
  
  writer = SummaryWriter()

  dict_hist = {
      'Train_acc' : [],
      'Train_loss' : [],
      'Valid_acc' : [],
      'Valid_loss': [],
      'Time_train': []
  }
  
  # Gradient scaling helps prevent gradients with small magnitudes from 
  # flushing to zero (“underflowing”) when training with mixed precision.
  scaler = amp.GradScaler()

  if scaler is not None: 
    functional.reset_net(net)
    net.train().to(device)

    for epoch in range(epochs):
        print("Epoch {}:".format(epoch+1))

        # ------------- TRAIN SET ----------- # 
        train_loss = 0.
        train_acc = 0.
        train_samples = 0

        time_train_batch = timing.time()

        for img, label in load_bar(train_data_loader):
            optimizer.zero_grad()
            img = img.to(device)
            label = label.to(device)

            if onehot == True:
              label = F.one_hot(label, num_classes).float()

            with amp.autocast():
              if(hasattr(net, 'T') == False):
                out_fr = 0.
                for t in range(T):
                    if (convolution == False):
                      img = encoder(img)
                    out_fr += net(img)
                out_fr = out_fr / T   
              else:
                  if (convolution == False):
                    img = encoder(img)
                  out_fr = net(img)

              loss = criterion(out_fr, label)
              scaler.scale(loss).backward()
              scaler.step(optimizer)
              scaler.update()
            
            train_samples += label.numel()
            train_loss += loss.item() * label.numel()
            train_acc += (out_fr.argmax(1) == label).float().sum().item()

            functional.reset_net(net)

        time_end = timing.time()

        train_loss /= train_samples
        train_acc /= train_samples

        dict_hist['Train_acc'].append(train_acc)
        dict_hist['Train_loss'].append(train_loss)
        dict_hist['Time_train'].append(time_end - time_train_batch)

        # ------------- STORING TRAIN BATCH RESULTS ----------- # 
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)

        writer.flush()
        writer.close()
       

        # ------------- VALIDATION ----------- # 
        if(validation_step == True):
          net.eval()
          val_loss = 0.
          val_acc = 0.
          val_samples = 0

          with torch.no_grad():
              for img, label in validation_data_loader:
                  img = img.to(device)
                  label = label.to(device)

                  if onehot == True:
                    label = F.one_hot(label, num_classes).float()

                  with amp.autocast():
                    if(hasattr(net, 'T') == False):
                      out_fr = 0.
                      for t in range(T):
                          if (convolution == False):
                            img = encoder(img)
                          out_fr += net(img)
                      out_fr = out_fr / T   
                    else:
                        if (convolution == False):
                          img = encoder(img)
                        out_fr = net(img)
                    
                  loss = criterion(out_fr, label)
                  val_samples += label.numel()
                  val_loss += loss.item() * label.numel()
                  val_acc += (out_fr.argmax(1) == label).float().sum().item()

                  functional.reset_net(net)

          val_loss /= val_samples
          val_acc /= val_samples

          dict_hist['Valid_acc'].append(val_acc)
          dict_hist['Valid_loss'].append(val_loss)

          # ------------- STORING VALID BATCH RESULTS ----------- # 
          writer.add_scalar(RUNS_PATH + '/Loss/val', val_loss, epoch)
          writer.add_scalar(RUNS_PATH + '/Accuracy/val', val_acc, epoch)

          writer.flush()
          writer.close()

        if(validation_step == True):
          print("B train acc: {:.4f} -- B train Loss: {:.4f} ## B valid acc: {:.4f} -- B valid loss: {:.4f}  ## Time: {:.2f}".format(train_acc, train_loss, val_acc, val_loss, (time_end - time_train_batch)))
        else:
          print("B train acc: {:.4f} -- B train Loss: {:.4f} ## Time: {:.2f}".format(train_acc, train_loss, (time_end - time_train_batch)))

    return dict_hist

  else:
    return print("Error in Scaler -> None")


In [ ]:
def val(net, device, data_loader, T=None):
    net.eval().to(device)
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for batch, (img, label) in enumerate(tqdm(data_loader)):
            img = img.to(device)
            if T is None:
                out = net(img)
            else:
                for m in net.modules():
                    if hasattr(m, 'reset'):
                        m.reset()
                for t in range(T):
                    if t == 0:
                        out = net(img)
                    else:
                        out += net(img)
            correct += (out.argmax(dim=1) == label.to(device)).float().sum().item()
            total += out.shape[0]
        acc = correct / total
        print('Validating Accuracy: %.3f' % (acc))
    return acc

In [ ]:
def evaluate_snn(net, device, data_loader, T):
  
    net.eval().to(device)

    correct = 0.
    total = 0.

    with torch.no_grad():
        for (img, label) in load_bar(data_loader):
            img = img.to(device)
            out = 0.
            for t in range(T):
                    out += net(img)
                    correct += (out.argmax(dim=1) == label.to(device)).float().sum().item()
            total += out.shape[0]

            functional.reset_net(net)

    print("\nModel Accuracy: {:.4f}".format((correct/total)*100))

###ANN functions

In [ ]:
def evaluate_ann(model, device, data_loader):

  model.eval().to(device)
  total = 0
  correct = 0

  with torch.no_grad():
    for batch, (img,label) in enumerate(load_bar(data_loader)):
      img = img.to(device)
      label = label.to(device)

      output = model(img)

      correct += (output.argmax(dim=1) == label).float().sum().item()
      #_ , predicted = torch.max(output.data, 1)
      #total += label.size(0)
      #correct += (predicted == label).sum().item()


    total = total + output.shape[0]
    
    print("\nModel Accuracy: {:.4f}".format((correct/total)*100))
    #print(f'\nAccuracy of the network on the 10000 test images: {100 * correct // total} %')

###Report functions

In [ ]:
def short_report(name="emissions.csv"):
  
  df = pd.read_csv(EMISSION_PATH + '/' + name)

  # Emissions: Emissions as CO₂-equivalents [CO₂eq], in kg
  # ._energy: Energy in kW
  # energy_consumed: sum of cpu+gpu+ram in kW
  df = df[['run_id', 'emissions', 'cpu_energy', 'gpu_energy', 'ram_energy', 'energy_consumed','Multi_step',
           'Time_train','Last_acc','Last_loss','Valid_acc','Valid_loss']]

  return df

In [ ]:
def train_report(train_dict, net, has_validation = False, name="emissions.csv"):
  
  df = pd.read_csv(EMISSION_PATH + '/' + name)

  ncols = len(df.columns)
  nrow = len(df.index)-1
  if ncols == 29:
    df['Multi_step'] = False
    df['Time_train'] = None
    df['Last_acc'] = None
    df['Last_loss'] = None
    df['Valid_acc'] = None
    df['Valid_loss'] = None

  if(hasattr(net, 'T') == True):
    df.at[nrow,'Multi_step'] = True
  else:
    df.at[nrow,'Multi_step'] = False

  tot_time = round(np.sum(train_dict['Time_train']), 2)
  acc_last = round(train_dict['Train_acc'][-1], 3)
  loss_last = round(train_dict['Train_loss'][-1], 3)

  df.at[nrow,'Time_train'] = tot_time
  df.at[nrow,'Last_acc']  = acc_last
  df.at[nrow,'Last_loss']  = loss_last

  if (has_validation == True):
      acc_last = round(train_dict['Valid_acc'][-1], 3)
      loss_last = round(train_dict['Valid_loss'][-1], 3)

      df.at[nrow,'Valid_acc']  = acc_last
      df.at[nrow,'Valid_loss']  = loss_last
  else:

      df.at[nrow,'Valid_acc']  = None
      df.at[nrow,'Valid_loss']  = None

  df.to_csv(EMISSION_PATH + '/' + name)


###Model construction

In [ ]:
###
#Standard CNN
###
def first_linear_input(input_dim,dict_cnn):
  h_out = input_dim
  for i in range(len(dict_cnn['filter_size'])):
    h_out = np.floor((h_out-(2-1)-1)/2+1)
  t_d = h_out**2*dict_cnn['n_filters'][-1]
  return int(t_d)

def cnn_layer_builder(model,in_f,out_f,size,pool):

  model.append(nn.Conv2d(in_f, out_f, kernel_size=size, padding='same'))
  model.append(nn.BatchNorm2d(out_f))
  model.append(nn.ReLU())
  if pool == 'maxpool':
    model.append(nn.MaxPool2d(2,2))
  return model

def lin_layer_builder(model,s_in,s_out,act='relu'):
  model.append(nn.Linear(s_in, s_out))
  if act == 'relu':
    model.append(nn.ReLU())
  elif act == 'out':
    pass
  return model

def init_cnn(cnn_d):
  filter_sizes = cnn_d['filter_size']
  n_filters = cnn_d['n_filters']

  l_sizes = cnn_d['linear_size']
  l_acts = cnn_d['lin_act']

  model = nn.Sequential()
  for k,j in zip( range(len(filter_sizes)) , range(1,len(n_filters))):
    model = cnn_layer_builder(model,n_filters[j-1], n_filters[j], filter_sizes[k], 'maxpool')
  model.append(nn.Flatten())
  for j,i in zip( range(len(l_acts)), range(1, len(l_sizes)) ):
    model = lin_layer_builder(model, l_sizes[i-1], l_sizes[i], l_acts[j])
            


  return model

In [ ]:
###
#CSNN
###

def first_linear_input(input_dim,dict_scnn):
  h_out = input_dim
  for p in dict_scnn['type_pool']:
    if p != 'no':
      h_out = h_out//2
  return dict_scnn['n_filters'][-1]*h_out**2

def scnn_layer_builder(model,in_f,out_f,size,pool,surr_f):

  model.append(layer.Conv2d(in_f, out_f, kernel_size=size, padding=(size//2,size//2), bias=False))
  model.append(layer.BatchNorm2d(out_f))
  if surr_f=='atan':
    model.append(neuron.IFNode(surrogate_function=surrogate.ATan()),)
  if pool == 'maxpool':
    model.append(layer.MaxPool2d(2,2))
  return model

def slin_layer_builder(model,s_in,s_out,surr_f):
  model.append(layer.Linear(s_in, s_out, bias=False))
  if surr_f == 'atan':
    model.append(neuron.IFNode(surrogate_function=surrogate.ATan()),)
  return model

###STDP functions

In [ ]:
def init_optimizers(net, istances, backprop_opt='adam', stdp_opt='sgd'):
  stdp_learners = []
  params_stdp = []
  params_gd = []

  layers = tuple(net.children())[0]

  for i in range(net.conv_fc.__len__()):
    if isinstance(layers[i], instances):
      stdp_learners.append(
          learning.STDPLearner(step_mode='m', synapse=net.conv_fc[i],
                                sn=net.conv_fc[i+1], tau_pre=2, tau_post=100,
                                f_pre=f_weight, f_post=f_weight)
      )
  for m in net.modules():
    if isinstance(m, instances):
      for p in m.parameters():
        params_stdp.append(p)

  params_stdp_set = set(params_stdp)
  params_gradient_descent = []
  for p in net.parameters():
    if p not in params_stdp_set:
      params_gd.append(p)

  if backprop_opt == 'adam':
    optimizer_gd = torch.optim.Adam(params_gd, lr=lr)
  if stdp_opt == 'sgd':
    optimizer_stdp = torch.optim.SGD(params_stdp, lr=lr)

  return optimizer_gd, optimizer_stdp, stdp_learners

In [ ]:
def fit_stdp(net, epochs, train_data_loader, device):
  dict_hist = {
      'Train_acc' : [],
      'Train_loss' : [],
      'Valid_acc' : [],
      'Valid_loss': [],
      'Time_train': []
  }

  writer = SummaryWriter('/content/runs')

  # Gradient scaling helps prevent gradients with small magnitudes from 
  # flushing to zero (“underflowing”) when training with mixed precision.
  scaler = amp.GradScaler()

  if scaler is not None: 
    functional.reset_net(net)

    for epoch in range(epochs):
        print("Epoch {}:".format(epoch+1))

        # ------------- TRAIN SET ----------- # 
        net.train()
        train_loss = 0
        train_acc = 0
        train_samples = 0

        time_train_batch = timing.time()


        for img, label in load_bar(train_data_loader):
            optimizer_gd.zero_grad()
            optimizer_stdp.zero_grad()
            img = img.to(device)
            label = label.to(device)
            label_onehot = F.one_hot(label, 10).float()

            #with amp.autocast():
            out_fr = net(img)

            loss = F.mse_loss(out_fr, label_onehot)
            #scaler.scale(loss).backward()
            loss.backward()

            optimizer_stdp.zero_grad()

            for i in range(stdp_learners.__len__()):
              stdp_learners[i].step(on_grad=True)

            #scaler.step(optimizer_gd)
            #scaler.step(optimizer_stdp)
            #scaler.update()
            optimizer_gd.step()
            optimizer_stdp.step()

            train_samples += label.numel()
            train_loss += loss.item() * label.numel()
            train_acc += (out_fr.argmax(1) == label).float().sum().item()

            functional.reset_net(net)
            del loss

        time_end = timing.time()

        train_loss /= train_samples
        train_acc /= train_samples

        dict_hist['Train_acc'].append(train_acc)
        dict_hist['Train_loss'].append(train_loss)
        dict_hist['Time_train'].append(time_end - time_train_batch)

        # ------------- STORING TRAIN BATCH RESULTS ----------- # 
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.flush()
        writer.close()
        print()
        print("B train acc: {:.4f} -- B train Loss: {:.4f} ## Time: {:.2f}".format(train_acc, train_loss, (time_end - time_train_batch)))

  return dict_hist

#Classes

In [ ]:
class SNN(nn.Module):

  def __init__(self, **kwargs):
      super().__init__()

      self.layer = nn.Sequential(
          layer.Flatten(),
          layer.Linear(28 * 28, 10, bias=False),
          neuron.IFNode(surrogate_function=surrogate.ATan()))
      
      if(len(kwargs) > 0):
        self.T = kwargs['T']
        functional.set_step_mode(self, step_mode="m")
        functional.set_backend(self, backend="cupy")

  def forward(self, x:torch.Tensor):
    if(hasattr(self, 'T')):
      x_seq = x.unsqueeze(0).repeat(self.T, 1, 1, 1, 1)
      x_seq = self.layer(x_seq)
      out = x_seq.mean(0)
    else:
      out = self.layer(x)
    return out

In [ ]:
class CSNN_cifar(nn.Module):

    def __init__(self, T: int, channels: int):
        super().__init__()
        self.T = T

        # torch.Size([3, 32, 32])
        self.conv_fc = nn.Sequential(
            
          #the image-spike encoding is implemented by the first three layers of the network
          layer.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False),
          layer.BatchNorm2d(channels),
          neuron.IFNode(surrogate_function=surrogate.ATan()),  

          layer.Conv2d(channels, 16, kernel_size=3, padding=1,bias=False),
          layer.Conv2d(16, 16, kernel_size=3, padding=1,bias=False),
          layer.BatchNorm2d(16),
          neuron.IFNode(surrogate_function=surrogate.ATan()),

          layer.Conv2d(16, 32, kernel_size=3, padding=1,bias=False),
          layer.Conv2d(32, 32, kernel_size=3, padding=1,bias=False),
          layer.BatchNorm2d(32),
          neuron.IFNode(surrogate_function=surrogate.ATan()),
          layer.AvgPool2d(2,2), #32 -> 16
          
          layer.Flatten(),
          layer.Linear(32 * 16 * 16, 512, bias=False),
          neuron.IFNode(surrogate_function=surrogate.ATan()),

          layer.Linear(512, 128, bias=False),
          neuron.IFNode(surrogate_function=surrogate.ATan()),

          layer.Linear(128, 10, bias=False),
          neuron.IFNode(surrogate_function=surrogate.ATan()))
  
        functional.set_step_mode(self, step_mode="m")
        functional.set_backend(self, backend="cupy")

    def forward(self, x: torch.Tensor):
      x_seq = x.unsqueeze(0).repeat(self.T, 1, 1, 1, 1)  # [N, C, H, W] -> [T, N, C, H, W]
      x_seq = self.conv_fc(x_seq)
      fr = x_seq.mean(0)
      return fr


In [ ]:
class CSNN_d(nn.Module):

    def __init__(self, dict_snn, backend="cupy",):
      super().__init__()
      self.T = dict_snn['T']
      self.conv_fc = self.build_net(dict_snn)

      functional.set_step_mode(self, step_mode="m")
      functional.set_backend(self, backend="cupy")
        
    def build_net(self,dict_snn):
      model = nn.Sequential()
      for k,j in zip( range(len(dict_snn['filter_size'])) , range(1,len(dict_snn['n_filters']))):
        model = scnn_layer_builder(model, dict_snn['n_filters'][j-1], dict_snn['n_filters'][j], dict_snn['filter_size'][k], dict_snn['type_pool'][k],dict_snn['surrogate'])
      model.append(layer.Flatten())
      for i in range(1, len(dict_snn['linear_size'])):
        model = slin_layer_builder(model, dict_snn['linear_size'][i-1], dict_snn['linear_size'][i],dict_snn['surrogate'])

      return model

    def forward(self, x: torch.Tensor):
      x_seq = x.unsqueeze(0).repeat(self.T, 1, 1, 1, 1)  # [N, C, H, W] -> [T, N, C, H, W]
      x_seq = self.conv_fc(x_seq)
      fr = x_seq.mean(0)
      return fr

#ANN to SNN convertion

In [ ]:
batch_size = 128
train_data_loader = init_data_loader(train_dataset, batch_size)

model_converter = ann2snn.Converter(mode='max', dataloader = train_data_loader)
snn_model = model_converter(cnn)

In [ ]:
test_data_loader = init_data_loader(test_dataset, batch_size)

In [ ]:
T = 50
mode_max_accs = evaluate_snn(snn_model, device, test_data_loader, T=T)
print('SNN accuracy (simulation %d time-steps): %.4f' % (T, mode_max_accs[-1]))

#Pretrained models

In [ ]:
all_models = tv_models.list_models()
classification_models = tv_models.list_models(module=tv_models)

In [ ]:
vgg_ls = []
resnet_ls = []

for i in classification_models:
  if "vgg" in i:
    vgg_ls.append(i)
  if "resnet" in i:
    resnet_ls.append(i)

In [ ]:
vgg_ls

['vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn']

In [ ]:
s_vgg19 = spiking_vgg.spiking_vgg19_bn(pretrained=True,
                                      spiking_neuron=neuron.IFNode, 
                                      surrogate_function=surrogate.ATan())

s_resnet50 = spiking_resnet.spiking_resnet50(pretrained=True,
                                          spiking_neuron=neuron.IFNode, 
                                          surrogate_function=surrogate.ATan())

s_vgg19.to(device)
s_resnet50.to(device)

In [ ]:
res = evaluate_snn(s_vgg19, device, imgnet_vgg19_loader, T = 50)

100%|██████████| 15/15 [03:30<00:00, 14.07s/it]

Validating Accuracy: 0.000


#TEST

In [ ]:
torch.cuda.empty_cache()

In [ ]:
spike_net_cifar = CSNN_cifar(T=20, channels=3)

In [ ]:
writer = SummaryWriter()

epochs = 30
batch_size = 128
lr = 1e-2

# Use Adam optimizer
optimizer = torch.optim.Adam(spike_net_cifar.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# Init data loaders
train_data_loader_cifar = init_data_loader(train_cifar, batch_size)
val_data_loader_cifar = init_data_loader(val_cifar, batch_size)

train_data_loader_cifar.num_workers=2

In [ ]:
'''
def fit_and_eval(net, epochs, optimizer, criterion, num_classes, 
                 train_data_loader, validation_data_loader, device,
                 T, validation_step = False, convolution = True, onehot = False):
'''

train_val_ls = fit_and_eval(spike_net_cifar, epochs, optimizer, criterion, cifar_nclasses,
                            train_data_loader_cifar, val_data_loader_cifar, device, 
                            T = spike_net_cifar.T, validation_step = False, convolution = True, onehot=False)